In [3]:
%pip install cvxpy

  Using cached clarabel-0.9.0-cp37-abi3-win_amd64.whl.metadata (4.8 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.1 MB 3.1 MB/s eta 0:00:01
   ----------------------------------- ---- 0.9/1.1 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 7.4 MB/s eta 0:00:00
Using cached clarabel-0.9.0-cp37-abi3-win_amd64.whl (736 kB)
   ---------------------------------------- 0.0/72.2 kB ? eta -:--:--
   ---------------------------------------- 72.2/72.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/293.4 kB ? eta -:--:--
   --------------------------------------- 293.4/293.4 kB 17.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/8.4 MB 40.3 MB/s eta 0:00:01
   -------------- ------------------------- 3.1/8.4 MB 39.5 MB/s eta 0:00:01
   ---------------------- ----------------- 4.8/8.4 MB 


[notice] A new release of pip is available: 23.3.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cvxpy as cp
import pandas as pd

file_path = r'..\data\TestingData.csv'
df = pd.read_csv(file_path)

# Define the variable for suppressed counts
suppressed_counts = cp.Variable(df.shape[0])

# Define the objective function (e.g., sum of squares of suppression amounts)
objective = cp.Minimize(cp.sum_squares(suppressed_counts - df['GraduationCount']))

# Define the privacy constraints
# For example, ensure that suppressed counts are above a threshold (e.g., 10)
constraints = [suppressed_counts >= 10]

# Set up constraints, knowing how the information works
# Subtotals
# Male + Female = Total students
# School 1 + school 2 ... = Total students
# Constraint for all values

# Suppress N size and track these

# Update constraints as table is being updated with suppressed counts
# Lea 1, School 1, male <= 10

# Maximize value of suppressed cell
# Lea 1, School 1, female == 50 


# Formulate and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Update the DataFrame with the suppressed counts
df['suppressed_count'] = suppressed_counts.value

print(df)


In [16]:
import cvxpy as cp
import pandas as pd

file_path = r'..\data\TestingData.csv'
df = pd.read_csv(file_path)

# Define the variable for suppressed counts
suppressed_counts = cp.Variable(df.shape[0])

# Define binary variables for each record to indicate if it needs complementary suppression
complementary_suppression = cp.Variable(df.shape[0], boolean=True)

# Define the objective function (e.g., sum of squares of suppression amounts)
objective = cp.Minimize(cp.sum_squares(suppressed_counts - df['GraduationCount']))

# Define the privacy constraints
constraints = [suppressed_counts >= 10]

df['ComboID'] = df['Subgroup1'].astype(str) + '_' + df['Subgroup2']
# Group by ComboID and add complementary suppression constraints
for combo_id, group in df.groupby('ComboID'):
    indices = group.index.tolist()
    if len(indices) > 1:
        # For simplicity, we're just ensuring the sum of counts in each group is above a threshold
        # This is a placeholder; the actual constraint logic will depend on your specific requirements
        constraints += [cp.sum(suppressed_counts[indices]) >= 10]

df['ComboID'] = df['ChildEntity'] + '_' + df['Subgroup1'].astype(str) + '_' + df['Subgroup2']
# Group by ComboID and add complementary suppression constraints
for combo_id, group in df.groupby('ComboID'):
    indices = group.index.tolist()
    if len(indices) > 1:
        # For simplicity, we're just ensuring the sum of counts in each group is above a threshold
        # This is a placeholder; the actual constraint logic will depend on your specific requirements
        constraints += [cp.sum(suppressed_counts[indices]) >= 10]

df['ComboID'] = df['ParentEntity'] + '_' + df['Subgroup1'].astype(str) + '_' + df['Subgroup2']
# Group by ComboID and add complementary suppression constraints
for combo_id, group in df.groupby('ComboID'):
    indices = group.index.tolist()
    if len(indices) > 1:
        # For simplicity, we're just ensuring the sum of counts in each group is above a threshold
        # This is a placeholder; the actual constraint logic will depend on your specific requirements
        constraints += [cp.sum(suppressed_counts[indices]) >= 10]

# Add constraints for complementary suppression
# Example: If records 1 and 2 together could reveal suppressed values, add a constraint
# This is a placeholder for the logic you'd need to implement based on your data
# constraints.append(complementary_suppression_logic)

# Formulate and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Update the DataFrame with the suppressed counts
df['suppressed_count'] = suppressed_counts.value

print(df[['ParentEntity', 'ChildEntity', 'Subgroup1', 'Subgroup2','GraduationCount', 'suppressed_count']])

   ParentEntity ChildEntity                     Subgroup1   Subgroup2  \
0     District1     School1      English Language Learner        Male   
1     District1     School1  Not English Language Learner        Male   
2     District1     School1      English Language Learner      Female   
3     District1     School1  Not English Language Learner      Female   
4     District1     School1      English Language Learner  Non-binary   
5     District1     School1  Not English Language Learner  Non-binary   
6     District1     School2      English Language Learner        Male   
7     District1     School2  Not English Language Learner        Male   
8     District1     School2      English Language Learner      Female   
9     District1     School2  Not English Language Learner      Female   
10    District1     School2      English Language Learner  Non-binary   
11    District1     School2  Not English Language Learner  Non-binary   
12    District2     School3      English Language L